In [1]:
# %% [markdown]
# ## 1) 환경 & 안전 설정

# %%
import os, random, math, time, json, re
import numpy as np
import torch

os.environ["TORCH_COMPILE_DISABLE"] = "1"  # 토치 컴파일 비활성

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16
elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
    device = torch.device("mps"); torch_dtype = torch.float16
else:
    device = torch.device("cpu"); torch_dtype = torch.float32

print(f"Device: {device}, dtype: {torch_dtype}")


Device: cuda, dtype: torch.bfloat16


In [2]:
# %% [markdown]
# ## 2) 모델 & 토크나이저 로드

# %%
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_ID = "openai/gpt-oss-20b"
REVISION = None

_tok = AutoTokenizer.from_pretrained(MODEL_ID, revision=REVISION, trust_remote_code=True, use_fast=True)
if _tok.pad_token is None:
    _tok.pad_token = _tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    revision=REVISION,
    trust_remote_code=True,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=False,
    device_map=None,
)
model.to(device); model.eval()

_gen = pipeline("text-generation", model=model, tokenizer=_tok,
                device=0 if device.type=="cuda" else -1)

print("Model & tokenizer ready.")


c:\Users\Admin\anaconda3\envs\gptoss\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
You have loaded an FP4 model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model. To remove this warning, pass device_map = 'cuda'. 
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.53s/it]
Device set to use cuda:0


Model & tokenizer ready.


In [3]:
# %% [markdown]
# ## 3) 짧은 구동 체크

# %%
TEST_PROMPT = [{"role": "user", "content": "안녕! 테스트야."}]
GEN_KW = dict(max_new_tokens=40, do_sample=False,
              eos_token_id=_tok.eos_token_id, pad_token_id=_tok.pad_token_id,
              return_full_text=False)

out = _gen(TEST_PROMPT, **GEN_KW)
print(out)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[{'generated_text': 'analysisThe user says "안녕! 테스트야." which is Korean: "Hello! This is a test." They likely want a friendly response. The instruction: "You are ChatGPT'}]


In [4]:
# %% [markdown]
# ## 4) emotion_vector

# %%
EMO_SYSTEM = (
    "You are EmotionScore. Respond with JSON ONLY. "
    "Return exactly these four keys as floats in [0,1]: "
    "dopamine, serotonin, norepinephrine, melatonin. "
    "Do not write any other text, no explanation."
)

def _clip01(x):
    try: return max(0.0, min(1.0, float(x)))
    except: return 0.0

def emotion_vector(text: str):
    msgs = [
        {"role": "system", "content": EMO_SYSTEM},
        {"role": "user", "content": f"Text: {text}"},
        {"role": "assistant", "content": "{"},
    ]
    out = _gen(msgs, max_new_tokens=60, do_sample=False,
               return_full_text=False,
               eos_token_id=_tok.eos_token_id, pad_token_id=_tok.pad_token_id)
    raw = out[0].get("generated_text","")
    import re, json
    m = re.search(r"\{.*\}", "{" + raw, re.S)
    data = json.loads(m.group())
    vec = [_clip01(data.get(k)) for k in ("dopamine","serotonin","norepinephrine","melatonin")]
    return vec, data

print(emotion_vector("우울하다"))


([0.2, 0.3, 0.4, 0.6], {'dopamine': 0.2, 'serotonin': 0.3, 'norepinephrine': 0.4, 'melatonin': 0.6})
